<a href="https://colab.research.google.com/github/NeuroWinter/news-headline-ai/blob/master/Spacy_transfomers_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U spacy[cuda100]
!pip install spacy-transformers
!python3 -m spacy download en_trf_distilbertbaseuncased_lg
!pip install torch

In [2]:
import pandas as pd
import csv

# REMEBER TO UPLOAD THIS!
input_file = 'news_headlines.csv'

TRAIN_DATA = []
df = pd.read_csv(input_file)

list_of_sites = list(df['news_site'].unique())

for row in df.iterrows():
    row_headline = row[1][2].strip()
    row_site     = row[1][1]
    if row_headline:
        # Add all the sites to the dict
        row_cat            = {site: 0.0 for site in list_of_sites}
        row_cat[row_site] += 1.0
        TRAIN_DATA.append((row_headline.strip(), {"cats": row_cat}))
print(TRAIN_DATA[0])
df = None

('Experts say adapting to climate change can pay off manifold', {'cats': {'ABC - Tech': 1.0, 'ABC - Top Stories': 0.0, 'ABC - US': 0.0, 'ABC - World': 0.0, 'Al Jazeera': 0.0, 'AllAfrica News - Mozambique': 0.0, 'ARRL': 0.0, 'Ars Technica - all features': 0.0, 'Ars Technica - all news': 0.0, 'Axios - Business': 0.0, 'Axios - Energy': 0.0, 'Axios - Future': 0.0, 'Axios - Health Care': 0.0, 'Axios - Politics': 0.0, 'Axios - Science': 0.0, 'Axios - Technology': 0.0, 'Axios - Top': 0.0, 'Axios - World': 0.0, 'Bandladesh - bdnews24.com': 0.0, 'Bangkok Post - Lifestyle': 0.0, 'Bangkok Post - Most Recent': 0.0, 'Bangkok Post - Top Stories': 0.0, 'Bangkok Post - Travel': 0.0, 'BBC': 0.0, 'BBC - Africa': 0.0, 'BBC - Asia': 0.0, 'BBC - England': 0.0, 'BBC - Europe': 0.0, 'BBC - Latin America': 0.0, 'BBC - Middle East': 0.0, 'BBC - Northern Ireland': 0.0, 'BBC - Scotland': 0.0, 'BBC - UK': 0.0, 'BBC - US & Canada': 0.0, 'BBC - Wales': 0.0, 'BBC - World': 0.0, 'Breitbart "News"': 0.0, 'Buzzfeed - U

In [0]:
# RUN THIS TO MAKE SURE THAT SPACY IS WORKING
import spacy

nlp = spacy.load('en_trf_distilbertbaseuncased_lg')
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")
print(doc[0].similarity(doc[7]))
print(doc._.trf_last_hidden_state.shape)

In [0]:
TRAIN_DATA = [
    ("text1", {"cats": {"POSITIVE": 1.0, "NEGATIVE": 0.0, "TESTING": 0.0}})
]

In [3]:
import spacy
import random
import torch

from spacy.util         import minibatch
from spacy_transformers import TransformersLanguage, TransformersWordPiecer, TransformersTok2Vec

BATCH_SIZE = 1000
ITERATIONS = 2
MODEL      = "distilbert-base-uncased"

is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    print("USING GPU")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

print(f"Number of lables: {len(list_of_sites)}")
print(f"Number of training data: {len(TRAIN_DATA)}")

nlp = spacy.load("en_trf_distilbertbaseuncased_lg")
print(nlp.pipe_names) # ["sentencizer", "trf_wordpiecer", "trf_tok2vec"]
textnews = nlp.create_pipe("trf_textcat", config={"exclusive_classes": True})
for label in list_of_sites:
    textnews.add_label(label)
nlp.add_pipe(textnews)
optimizer = nlp.resume_training()
for i in range(ITERATIONS):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for x, batch in enumerate(minibatch(TRAIN_DATA, size=BATCH_SIZE)):
        texts, news = zip(*batch)
        nlp.update(texts, news, sgd=optimizer, losses=losses)
        print(f"Batch number: {x}, Losses: {losses}")
    print(f"Iteration: {i}, Losses: {losses}")

USING GPU
Number of lables: 450
Number of training data: 240403
['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']
Batch number: 0, Losses: {'trf_textcat': 9.977775334846228e-07}
Batch number: 1, Losses: {'trf_textcat': 1.9948396356994635e-06}
Batch number: 2, Losses: {'trf_textcat': 2.9914148171883426e-06}
Batch number: 3, Losses: {'trf_textcat': 3.992650817963295e-06}
Batch number: 4, Losses: {'trf_textcat': 4.993356014892925e-06}
Batch number: 5, Losses: {'trf_textcat': 5.991116836412402e-06}
Batch number: 6, Losses: {'trf_textcat': 6.985108484514058e-06}
Batch number: 7, Losses: {'trf_textcat': 7.977493510225031e-06}
Batch number: 8, Losses: {'trf_textcat': 8.969859663920943e-06}
Batch number: 9, Losses: {'trf_textcat': 9.960341685655294e-06}
Batch number: 10, Losses: {'trf_textcat': 1.094969775294885e-05}
Batch number: 11, Losses: {'trf_textcat': 1.1938822581214481e-05}
Batch number: 12, Losses: {'trf_textcat': 1.292797492169484e-05}
Batch number: 13, Losses: {'trf_textcat': 1.39162

In [1]:
a = nlp("Parliament passes Zainab Alert Bill for recovery of missing children - Samaa News")
a.cats
for key, value in sorted(a.cats.items(), key=lambda item: item[1]):
    print("%s: %s" % (key, value))

NameError: ignored

In [0]:
from spacy_transformers import TransformersLanguage, TransformersWordPiecer, TransformersTok2Vec

name = "bert-base-uncased"
nlp = TransformersLanguage(trf_name=name, meta={"lang": "en"})
nlp.add_pipe(nlp.create_pipe("sentencizer"))
nlp.add_pipe(TransformersWordPiecer.from_pretrained(nlp.vocab, name))
nlp.add_pipe(TransformersTok2Vec.from_pretrained(nlp.vocab, name))
print(nlp.pipe_names)  # ['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']

In [0]:
textnews = nlp.create_pipe("trf_textcat", config={"exclusive_classes": True})
for label in list_of_sites:
    textnews.add_label(label)
nlp.add_pipe(textnews)